In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q bertopic[all] sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 15.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import torch
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load data
df = pd.read_csv("/content/drive/MyDrive/review_sample.csv")
df = df[['text', 'stars']].dropna()
df['sentiment_label'] = df['stars'].apply(lambda x: 'positive' if x >= 4 else ('negative' if x <= 2 else 'neutral'))



In [ ]:
# Sample data for topic modeling
sampled_df = df.sample(100000, random_state=42)
docs = sampled_df['text'].tolist()

In [ ]:
# Load sentence transformer for LLM-based embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize BERTopic with custom embedding model
topic_model = BERTopic(embedding_model=embedding_model, verbose=True)

# Fit and transform
topics, probs = topic_model.fit_transform(docs)

# Add topics back to dataframe
sampled_df["topic"] = topics

# Show top topics
topic_info = topic_model.get_topic_info()
print(topic_info.head())

# Show example documents from a few topics
for topic_id in topic_info['Topic'].head(5):
    print(f"\nTopic {topic_id}:")
    print(topic_model.get_topic(topic_id))


2025-03-30 02:42:27,754 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

2025-03-30 02:44:52,661 - BERTopic - Embedding - Completed ✓
2025-03-30 02:44:52,664 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-30 02:47:28,724 - BERTopic - Dimensionality - Completed ✓
2025-03-30 02:47:28,728 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-30 02:47:42,148 - BERTopic - Cluster - Completed ✓
2025-03-30 02:47:42,176 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-03-30 02:47:49,886 - BERTopic - Representation - Completed ✓


   Topic  Count                             Name  \
0     -1  42735               -1_was_food_we_the   
1      0   2632  0_food_great_service_atmosphere   
2      1   2270          1_sushi_roll_rolls_tuna   
3      2   2161    2_pizza_crust_pizzas_delivery   
4      3   1967          3_hotel_room_stay_rooms   

                                      Representation  \
0  [was, food, we, the, were, it, and, had, with,...   
1  [food, great, service, atmosphere, friendly, s...   
2  [sushi, roll, rolls, tuna, hibachi, sashimi, j...   
3  [pizza, crust, pizzas, delivery, pepperoni, to...   
4  [hotel, room, stay, rooms, desk, bed, stayed, ...   

                                 Representative_Docs  
0  [Hate to do this to Villagio but I feel it nec...  
1  [I love this place!!! They have great service,...  
2  [I have tried this place 3 times now over the ...  
3  [We ordered from here for the first time a few...  
4  [Giving 5 stars for this hotel in this price r...  

Topic -1:
[('was', 

In [ ]:
# Sample data for topic modeling
sampled_df = df.sample(100000, random_state=42)
docs = sampled_df['text'].tolist()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Get c-TF-IDF matrix (topic-word distributions)
ctfidf = topic_model.c_tf_idf_

# Calculate pairwise cosine similarity
cos_sim = cosine_similarity(ctfidf)

# Mask diagonal (self-similarity)
np.fill_diagonal(cos_sim, 0)

# Mean similarity across different topics
mean_similarity = np.mean(cos_sim)

# Invert to interpret as "coherence" (lower similarity = more distinct = better)
coherence_score = 1 - mean_similarity

print("Approximate Topic Coherence Score (inverted cosine sim):", coherence_score)


Approximate Topic Coherence Score (inverted cosine sim): 0.9135639298760588


In [ ]:
# Visualize topic clusters in 2D space
topic_model.visualize_documents(docs)

# Visualize most frequent topics
topic_model.visualize_barchart(top_n_topics=10)


In [ ]:
# STEP 1: Install required packages
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# STEP 2: Load dataset
df = pd.read_csv("/content/drive/MyDrive/cleaned_reviews.csv")
df = df[['cleaned_text', 'stars']].dropna()

# STEP 3: Create sentiment labels (set use_neutral=False for binary classification)
use_neutral = True  # Set to False if you want only positive vs negative

if use_neutral:
    df['sentiment_label'] = df['stars'].apply(lambda x: 'positive' if x >= 4 else ('negative' if x <= 2 else 'neutral'))
else:
    df = df[df['stars'] != 3]  # Drop neutral reviews
    df['sentiment_label'] = df['stars'].apply(lambda x: 'positive' if x >= 4 else 'negative')

# Encode labels (optional)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment_label'])


In [ ]:
# Downsample classes to handle imbalance
df_majority = df[df.label == 2]  # positive
df_minority = df[df.label != 2]

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=42)

df_balanced = pd.concat([df_majority, df_minority_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)
print(df_balanced['label'].value_counts())


label
2    343321
0    228202
1    115119
Name: count, dtype: int64


In [ ]:
device = 0 if torch.cuda.is_available() else -1
model_name = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# STEP 5: Create classification pipeline with truncation enabled
classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=device,
    truncation=True,
    padding=True,
    max_length=512,
    return_all_scores=False
)

# STEP 6: Batch prediction
batch_size = 64
reviews = df['cleaned_text'].tolist()
predictions = []
total_batches = len(reviews) // batch_size + int(len(reviews) % batch_size != 0)

for i in range(0, len(reviews), batch_size):
    batch = reviews[i:i + batch_size]

    # Tokenizer will handle truncation and padding correctly
    results = classifier(batch)
    predictions.extend([res['label'] for res in results])

    if (i // batch_size) % 100 == 0:
        print(f"Processed {i // batch_size}/{total_batches} batches...")

# STEP 7: Map model labels to dataset labels
label_map = {
    "LABEL_0": "negative",
    # "LABEL_1": "neutral",   # If use_neutral = False, remove this
    "LABEL_2": "positive"
}
df['predicted_sentiment'] = [label_map.get(label, 'neutral') for label in predictions]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Processed 0/6991 batches...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 100/6991 batches...
Processed 200/6991 batches...
Processed 300/6991 batches...
Processed 400/6991 batches...
Processed 500/6991 batches...
Processed 600/6991 batches...
Processed 700/6991 batches...
Processed 800/6991 batches...
Processed 900/6991 batches...
Processed 1000/6991 batches...
Processed 1100/6991 batches...
Processed 1200/6991 batches...
Processed 1300/6991 batches...
Processed 1400/6991 batches...
Processed 1500/6991 batches...
Processed 1600/6991 batches...
Processed 1700/6991 batches...
Processed 1800/6991 batches...
Processed 1900/6991 batches...
Processed 2000/6991 batches...
Processed 2100/6991 batches...
Processed 2200/6991 batches...
Processed 2300/6991 batches...
Processed 2400/6991 batches...
Processed 2500/6991 batches...
Processed 2600/6991 batches...
Processed 2700/6991 batches...
Processed 2800/6991 batches...
Processed 2900/6991 batches...
Processed 3000/6991 batches...
Processed 3100/6991 batches...
Processed 3200/6991 batches...
Processed 3300/69

In [ ]:
df_clean = df[df['sentiment_label'].isin(['positive', 'negative'])]


In [ ]:
# Compute Accuracy
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(df_clean['sentiment_label'], df_clean['predicted_sentiment'])
print(f"Model Accuracy: {accuracy:.4f}")

# Generate Classification Report
print("Classification Report:")
print(classification_report(df_clean['sentiment_label'], df_clean['predicted_sentiment']))


Model Accuracy: 0.8366
Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

    negative       0.96      0.49      0.65    104088
     neutral       0.00      0.00      0.00         0
    positive       0.94      0.94      0.94    343321

    accuracy                           0.84    447409
   macro avg       0.63      0.48      0.53    447409
weighted avg       0.94      0.84      0.87    447409



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
df_sample = df.sample(n=100000, random_state=42).reset_index(drop=True)

# STEP 5: Load Karimbkh fine-tuned Yelp model
device = 0 if torch.cuda.is_available() else -1
model_name = "karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# STEP 6: Create classification pipeline with truncation
classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=device,
    truncation=True,
    padding=True,
    max_length=512,
    return_all_scores=False
)

# STEP 7: Perform batch inference
batch_size = 64
reviews = df_sample['cleaned_text'].tolist()
predictions = []
total_batches = len(reviews) // batch_size + int(len(reviews) % batch_size != 0)

for i in range(0, len(reviews), batch_size):
    batch = reviews[i:i + batch_size]
    results = classifier(batch)
    predictions.extend([res['label'] for res in results])

    if (i // batch_size) % 50 == 0:
        print(f"Processed {i // batch_size}/{total_batches} batches...")

# STEP 8: Map model labels (LABEL_0, LABEL_1, LABEL_2) to sentiment
label_map = {
    "LABEL_0": "negative",
    # "LABEL_1": "neutral",
    "LABEL_2": "positive"
}
df_sample['predicted_sentiment'] = [label_map.get(label, 'neutral') for label in predictions]



Device set to use cuda:0


Processed 0/1563 batches...
Processed 50/1563 batches...
Processed 100/1563 batches...
Processed 150/1563 batches...
Processed 200/1563 batches...
Processed 250/1563 batches...
Processed 300/1563 batches...
Processed 350/1563 batches...
Processed 400/1563 batches...
Processed 450/1563 batches...
Processed 500/1563 batches...
Processed 550/1563 batches...
Processed 600/1563 batches...
Processed 650/1563 batches...
Processed 700/1563 batches...
Processed 750/1563 batches...
Processed 800/1563 batches...
Processed 850/1563 batches...
Processed 900/1563 batches...
Processed 950/1563 batches...
Processed 1000/1563 batches...
Processed 1050/1563 batches...
Processed 1100/1563 batches...
Processed 1150/1563 batches...
Processed 1200/1563 batches...
Processed 1250/1563 batches...
Processed 1300/1563 batches...
Processed 1350/1563 batches...
Processed 1400/1563 batches...
Processed 1450/1563 batches...
Processed 1500/1563 batches...
Processed 1550/1563 batches...


In [ ]:
df_sample['predicted_sentiment'] = df_sample['predicted_sentiment'].replace('neutral', 'positive')

In [ ]:
# STEP 9: Evaluation
accuracy = accuracy_score(df_sample['sentiment_label'], df_sample['predicted_sentiment'])
print(f"\n✅ Model Accuracy: {accuracy:.4f}\n")
print("🔍 Classification Report:")
print(classification_report(df_sample['sentiment_label'], df_sample['predicted_sentiment']))


✅ Model Accuracy: 0.9422

🔍 Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.86      0.87     23191
    positive       0.96      0.97      0.96     76809

    accuracy                           0.94    100000
   macro avg       0.92      0.91      0.92    100000
weighted avg       0.94      0.94      0.94    100000

